In [1]:
# !wget https://raw.githubusercontent.com/Teradata/kylo/master/samples/sample-data/parquet/userdata1.parquet
# !wget https://repo.maven.apache.org/maven2/org/apache/flink/flink-sql-parquet_2.11/1.12.2/flink-sql-parquet_2.11-1.12.2.jar

In [5]:
from pyflink.dataset import ExecutionEnvironment
from pyflink.table import TableConfig, DataTypes, BatchTableEnvironment, EnvironmentSettings
from pyflink.table.descriptors import Schema, OldCsv, FileSystem
from pyflink.table.types import DataTypes
from pyflink.table.expressions import col, lit

In [26]:
t_env = BatchTableEnvironment.create(
    environment_settings=EnvironmentSettings.new_instance()
    .in_batch_mode().use_blink_planner().build())

t_env._j_tenv.getPlanner().getExecEnv().setParallelism(1)

JavaObject id=o444

In [27]:
my_source_ddl = f"""
CREATE TABLE mySource (
  registration_dttm TIMESTAMP,
  id INT,
  first_name STRING,
  last_name STRING,
  email STRING,
  gender STRING,
  ip_address STRING,
  cc STRING,
  country STRING,
  birthdate STRING,
  salary DOUBLE,
  title STRING,
  comments STRING
) WITH (
 'connector' = 'filesystem',
 'path' = '/notebooks/userdata1.parquet',
 'format' = 'parquet'
)
"""

my_sink_ddl = f"""
CREATE TABLE mySink (
  registration_dttm TIMESTAMP(6),
  id INT,
  first_name STRING,
  last_name STRING,
  email STRING,
  gender STRING,
  ip_address STRING,
  cc STRING,
  country STRING,
  birthdate STRING,
  salary DOUBLE,
  title STRING,
  comments STRING
) WITH (
 'connector' = 'filesystem',
 'path' = '/notebooks/output',
 'format' = 'csv'
)
"""

t_env.sql_update(my_source_ddl)
t_env.sql_update(my_sink_ddl)

In [30]:
tab = t_env.from_path('mySource')
tab.select('*').insert_into('mySink')

t_env.execute('sink_parquet')

In [32]:
# !/opt/flink/bin/flink run --detached -py /notebooks/parquet.py 